<a href="https://colab.research.google.com/github/diegoeller/LLM---Marketing/blob/main/LLMs_para_empresas_e_neg%C3%B3cios_Marketing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto para Departamento de Marketing

Nosso objetivo com esse projeto é criar um assistente de geração de conteúdo automatizado, que adapta o texto ao público e ao canal de divulgação. Confira os slides para mais detalhes sobre a proposta desse estudo de caso.


> **Importante:** Caso dê algum erro no processo de instalação e que impeça de prosseguir com a execução do código, confira o Colab da aula e verifique se fez uma cópia do mais atualizado, pois atualizaremos essas etapas de instalação com os comandos atualizados (caso seja necessária alguma mudança no comando de instalação).

Vamos usar primeiro o ipynb no Colab para desenvolver e validar a lógica com LLMs, onde aprenderemos a deixar uma aplicação funcional dentro do próprio Colab usando ipywidgets. Ao final, veremos como adaptar isso para uma interface profissional usando o framework Streamlit, pronto para publicar. Isso evita retrabalho, ajuda a testar ideias com rapidez e foca primeiro no que importa: o núcleo funcional, a lógica e conceitos.



## Instalação das bibliotecas

Abaixo instalaremos algumas bibliotecas essenciais para o desenvolvimento de nosso projeto.

Para instalação usaremos o comando pip install. Passaremos o parâmetro -q (quiet) para reduzir a verbosidade da saída no terminal, exibindo apenas erros e mensagens essenciais. É usado para simplificar a visualização durante instalações automatizadas ou em ambientes onde logs detalhados não são necessários.



In [2]:
!pip install -q langchain langchain-community langchain-groq ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


### Importar bibliotecas  


In [3]:
from google.colab import widgets
import ipywidgets as widgets

from langchain_groq import ChatGroq
import os
import getpass

# Criação dos campos - Interface

Antes de partirmos para o código, é importante definirmos com clareza os campos que a aplicação irá utilizar. Essa etapa é essencial para evitar dispersão e garantir que o desenvolvimento seja focado nas necessidades reais da empresa.

> Campos *(conforme discutido na apresentação do estudo de caso)*
 * Plataforma de destino (ex: Blog, Instagram, LinkedIn, E-mail)
 * Tom da mensagem (ex: Informativo, Inspirador, Urgente, Informal)
 * Comprimento do texto (ex: Curto, Médio, Longo)
 * Tema ou tópico (ex: alimentação, saúde mental, exames de rotina, cuidados, etc.)
 * Público-alvo (Jovens adultos, Famílias, Idosos, Geral, etc.)
 * Opções adicionais:
  * Incluir chamada para ação (ex: “Agendar consulta” ou “Converse com um especialista”)
  * Retornar hashtags
  * Inserir palavras-chave para incluir no meio do texto


Vamos começar criando um campo em formato de texto. O `widgets.Text` cria um campo livre para digitação, onde o usuário insere o conteúdo manualmente.

* `description`: texto que aparece como rótulo do campo (ajuda a identificar sua função).
* `placeholder`: texto que aparece dentro do campo antes do preenchimento, como sugestão ou exemplo. Vamos aproveitar para colocar uma sugestão já do que o usuário pode digitar, o que é uma boa prática de user experience (UX)

Obs: Pensando nas boas práticas, também vamos aproveitar para definir os nomes das variáveis em inglês (tema vai ser *topic*, público-alvo vai ser *audience*, etc.)



In [4]:
topic = widgets.Text(
    description = 'Tema:',
    placeholder = 'Ex: saúde mental, alimentação saudável, prevenção, etc.'
)

### Exibindo o widget

Para exibir os campos/widgets que criamos vamos usar o método display(). Com isso o campo vai aparecer dentro da saída do bloco de código abaixo, assim exibindo tudo de forma interativa dentro desse notebook.

In [5]:
display(topic)

Text(value='', description='Tema:', placeholder='Ex: saúde mental, alimentação saudável, prevenção, etc.')

In [6]:
topic.value

''

### Ajustando propriedades do campo

Por padrão, o widget Text do ipywidgets cria um campo de entrada relativamente estreito, o que pode não ser ideal quando esperamos que o usuário digite frases ou trechos mais longos.

Com `layout=widgets.Layout(width='500px')` definimos explicitamente a largura do campo como 500 pixels, o que é mais apropriado quando esperamos frases completas.

* Você pode ajustar esse valor conforme a necessidade - ex: '100%' para ocupar toda a largura do container (deixando responsivo), ou '700px' para um campo ainda maior.


In [7]:
topic = widgets.Text(
    description = 'Tema:',
    placeholder = 'Ex: saúde mental, alimentação saudável, prevenção, etc.',
    layout = widgets.Layout(width='500px')
)
display(topic)

Text(value='', description='Tema:', layout=Layout(width='500px'), placeholder='Ex: saúde mental, alimentação s…

### Outros formatos de campos

Para adicionar campos de seleção práticos e dinâmicos à nossa aplicação, utilizaremos a função widgets.Dropdown, que exibe opções em formato de lista suspensa. Passaremos as escolhas disponíveis através do parâmetro options e, para otimizar a interface e facilitar futuras alterações, definiremos uma largura padrão para esses campos usando uma variável, permitindo ajustes globais de tamanho de forma simples, o que pode ser muito útil caso os valores pré-definidos sejam extensos.


In [8]:
w_dropdown = '250px'

platform = widgets.Dropdown(
    options = ['Instagram', 'Facebook', 'LinkedIn', 'Blog', 'E-mail'],
    description = 'Plataforma',
    layout = widgets.Layout(width = w_dropdown)
)

tone = widgets.Dropdown(
    options=['Normal', 'Informativo', 'Inspirador', 'Urgente', 'Informal'],
    description='Tom:',
    layout=widgets.Layout(width=w_dropdown)
)

length = widgets.Dropdown(
    options=['Curto', 'Médio', 'Longo'],
    description='Tamanho:',
    layout=widgets.Layout(width=w_dropdown)
)

audience = widgets.Dropdown(
    options=['Geral', 'Jovens adultos', 'Famílias', 'Idosos', 'Adolescentes'],
    description='Público-alvo:',
    layout=widgets.Layout(width=w_dropdown)
)


display(platform, tone, length, audience)

Dropdown(description='Plataforma', layout=Layout(width='250px'), options=('Instagram', 'Facebook', 'LinkedIn',…

Dropdown(description='Tom:', layout=Layout(width='250px'), options=('Normal', 'Informativo', 'Inspirador', 'Ur…

Dropdown(description='Tamanho:', layout=Layout(width='250px'), options=('Curto', 'Médio', 'Longo'), value='Cur…

Dropdown(description='Público-alvo:', layout=Layout(width='250px'), options=('Geral', 'Jovens adultos', 'Famíl…

In [9]:
platform.value

'Instagram'

Para incorporar opções de ativar/desativar funcionalidades, como incluir uma Chamada para Ação (CTA) ou solicitar sugestões de hashtags, utilizaremos widgets.Checkbox.

Estes campos booleanos (Verdadeiro/Falso) serão configurados com um valor inicial (por padrão, desmarcado) e uma descrição clara de sua função, permitindo ao usuário controlar facilmente aspectos específicos da geração de conteúdo.

In [10]:
cta = widgets.Checkbox(
    value = False,
    description = 'Incluir CTA'
)

hashtags = widgets.Checkbox(
    value=False,
    description='Retornar Hashtags',
)

In [11]:
display(cta)

Checkbox(value=False, description='Incluir CTA')

In [12]:
cta.value

False

Para permitir a inserção de textos mais longos, como listas de palavras-chave para SEO, implementaremos um campo do tipo Textarea. Este campo opcional dará ao usuário a flexibilidade de especificar termos que a IA deve incorporar naturalmente ao conteúdo, e seu tamanho pode ser ajustado em largura e altura para melhor acomodar o texto inserido, utilizando `widgets.Layout` para definir dimensões como height.

In [13]:
keywords = widgets.Textarea(
    description = 'Palavras-chave (SEO)',
    placeholder = 'Ex: bem-estar, medicina preventiva...',
    layout = widgets.Layout(width = '500px', height = '50px')
)

In [14]:
display(keywords)

Textarea(value='', description='Palavras-chave (SEO)', layout=Layout(height='50px', width='500px'), placeholde…

## Criando o botão de geração

Vamos agora adicionar um botão à interface. Esse botão será clicado para gerar o conteúdo com base nos campos preenchidos. O parâmetro description aqui é o texto que aparece no botão.


In [15]:
generate_button = widgets.Button(
    description = 'Gerar conteúdo',
)

display(generate_button)

Button(description='Gerar conteúdo', style=ButtonStyle())

## Exibição do resultado

Precisamos criar um espaço para exibir o output, que é o resultado gerado pela LLM.

Usamos o Output() para mostrar o resultado da geração de conteúdo. Ele cria uma “área de resposta”, onde vamos exibir o conteúdo gerado. Tudo que for mostrado com display() ou print() dentro dele aparecerá aqui.


In [16]:
output = widgets.Output()

### Definindo ação do botão

Por enquanto o botão não faz nada, precisamos criar uma função que será executada quando ele for clicado.

Explicando os parâmetros:

* `b` é o próprio botão sendo passado como argumento (padrão do on_click).

* `with output`: garante que tudo dentro desse bloco apareça na área de saída.

* `clear_output()` limpa o resultado anterior, evitando sobreposição de textos.


In [17]:
def generate_result(b):
  with output:
    output.clear_output()
    print("Ok!")

**Ligando o botão à função**

`.on_click()` define que nossa função será executada quando o botão for pressionado

In [18]:
generate_button.on_click(generate_result)

**Testando**

Execute o bloco de código abaixo e clique no botão para verificar se nosso método está funcionando.

In [19]:
display(generate_button, output)

Button(description='Gerar conteúdo', style=ButtonStyle())

Output()

## Exibindo os campos juntos na interface

Por fim, precisamos organizar os campos e exibi-los num layout final junto ao botão.
Antes de chamarmos a função display (para exibir tudo de forma interativa dentro desse notebook) vamos usar a função `VBox()`, para organizar os elementos na vertical, na ordem em que forem listados.

E para evitar a repetição, coloque dentro de uma função chamada "create_form", que retorne esse VBox com os widgets. Assim seu código fica mais limpo e reutilizável, pois usaremos mais tarde esses campos novamente

In [20]:
def create_form():
  return widgets.VBox([
      topic,
      platform,
      tone,
      length,
      audience,
      cta,
      hashtags,
      keywords,
      generate_button,
      output
  ])

form = create_form()

display(form)

# Conectando com a LLM

Para integrar a LLM à nossa aplicação, precisamos definir o modelo e a forma de implementação, que pode ser via download (para modelos open source, garantindo execução local e privacidade) ou através de API (simplificando a integração, oferecendo boa performance em qualquer máquina, mas com processamento de dados em servidores externos).

## Escolhendo o modelo

Na fase inicial de testes, recomenda-se começar com um modelo open source acessível via API gratuita, o que simplifica a implementação e reduz custos. Mesmo após a aplicação estar funcionando, esses modelos seguem vantajosos pela flexibilidade e economia. Neste curso, iniciaremos com o uso via API para evitar a complexidade da configuração local. Mais adiante, ensinaremos como rodar modelos localmente, permitindo que você compare as abordagens e escolha a mais adequada ao seu caso.

Usaremos a biblioteca LangChain para integrar com a Groq, aproveitando seu módulo nativo de conexão e os benefícios que ela oferece no desenvolvimento.

Para escolher bons modelos, recomendamos consultar leaderboards comparativos, como:
 * o https://lmarena.ai/?leaderboard
 * ou ranking específico para português na Hugging Face - https://huggingface.co/spaces/eduagarcia/open_pt_llm_leaderboard

**Adicionando a key**

Antes de começar com o código, você deve colar no campo a sua key gerada dentro do painel do Groq: https://console.groq.com/keys



In [21]:
os.environ["GROQ_API_KEY"] = getpass.getpass()

··········


Lembre-se que não precisamos pagar para usar modelos disponibilizados gratuitamente pelo provedor.

* Essa próxima linha usa o método ChatGroq para configuração do modelo via API do Groq.

* Escolhemos um modelo gratuito, dentro da aba *free tier* https://console.groq.com/docs/rate-limits. Copie o ID do modelo e adicione no campo a seguir



In [22]:
id_model = "llama-3.3-70b-versatile" #@param {type: "string"}

llm = ChatGroq(
    model = id_model,
    temperature = 0.7,
    max_tokens=None,
    timeout = None,
    max_retries = 2,
)

### Explicações do método

Para este teste definimos a temperatura como 0.7.
A temperatura é um hiperparâmetro que ajusta a aleatoriedade da resposta da LLM.

* Temperaturas mais altas (0.8-1.0) geram saídas mais criativas, ideais para brainstorming, enquanto temperaturas baixas (0.0-0.4) produzem respostas mais focadas e determinísticas, adequadas para tarefas técnicas;
* valores médios (0.5-0.7) oferecem um equilíbrio, sendo um bom ponto de partida para geração de conteúdo geral, embora seja recomendável experimentar diferentes valores conforme o objetivo e o modelo.
Além da temperatura, outros parâmetros como max_tokens (limite de tokens), timeout (tempo máximo de resposta) e max_retries (tentativas em caso de falha) podem ser configurados para otimizar o comportamento da LLM, com a documentação da LangChain para Groq oferecendo detalhes sobre todas as opções disponíveis.

https://python.langchain.com/api_reference/groq/chat_models/langchain_groq.chat_models.ChatGroq.html

### Formato das mensagens

Ao interagir com a LLM, estruturamos o prompt como uma troca de mensagens, cada uma com uma função (ou *role*, como "human" para nossa entrada e "system" para instruções gerais que garantem consistência) e um conteúdo (a mensagem em si, seja texto ou dados estruturados).

O prompt de sistema é crucial para definir o comportamento base da LLM, como atribuir um papel ou instruções padrão, e embora um prompt genérico possa funcionar, um prompt de sistema específico para a aplicação melhora significativamente a consistência dos resultados.


In [23]:
prompt = "Olá! Quem é você?" # @param {type:"string"}

template = [
    ("system", "Você é um redator profissional."),
    ("human", prompt)
]

res = llm.invoke(template)
res

AIMessage(content='Olá! Eu sou um assistente de linguagem treinado por máquina. Posso ser considerado um redator profissional virtual, pois posso criar textos, responder a perguntas e até mesmo ajudar a editar e melhorar textos existentes. Estou aqui para ajudar com qualquer coisa que você precise em termos de redação, comunicação ou até mesmo para bater um papo! Como posso ajudar você hoje?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 96, 'prompt_tokens': 51, 'total_tokens': 147, 'completion_time': 0.273570075, 'prompt_time': 0.00263807, 'queue_time': 0.220464757, 'total_time': 0.276208145}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_9e1e8f8435', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--bd9a7b82-7967-4475-b68e-02b2d4786329-0', usage_metadata={'input_tokens': 51, 'output_tokens': 96, 'total_tokens': 147})

In [24]:
prompt = "Olá! Quem é você?" # @param {type:"string"}

template = [
    ("system", "Você é um redator profissional."),
    ("human", prompt)
]

res = llm.invoke(template)
res

AIMessage(content='Olá! Eu sou um assistente virtual treinado por uma equipe de especialistas em linguagem e comunicação. Meu objetivo é fornecer informações precisas e úteis para ajudar as pessoas a encontrar respostas para suas perguntas e necessidades.\n\nComo um redator profissional, posso criar conteúdo de alta qualidade em uma variedade de estilos e formatos, desde artigos e blog posts até relatórios e apresentações. Estou aqui para ajudar com qualquer coisa que você precise, seja para criar conteúdo novo ou revisar e melhorar o que já existe.\n\nMinhas habilidades incluem:\n\n* Redação criativa e técnica\n* Edição e revisão de texto\n* Pesquisa e análise de informações\n* Criação de conteúdo para redes sociais e outros canais de comunicação\n\nEstou ansioso para saber como posso ajudar você! Qual é o seu projeto ou necessidade atual?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 203, 'prompt_tokens': 51, 'total_tokens': 254, 'completion_time': 0

In [31]:
res.content

'Olá! Sou um assistente de linguagem treinado por máquina, mas também posso ser considerado um redator profissional, pois posso criar textos de alta qualidade e personalizados de acordo com as necessidades dos meus usuários.\n\nTenho habilidades em redação, edição e revisão de textos, e posso ajudar com uma variedade de tarefas, como:\n\n* Criar conteúdo para sites, blogs e redes sociais\n* Redigir artigos, relatórios e documentos\n* Editar e revisar textos para garantir clareza, coerência e precisão\n* Desenvolver estratégias de comunicação eficazes\n* Traduzir textos para diferentes idiomas\n\nEstou aqui para ajudar e colaborar com você em seus projetos de redação e comunicação. Como posso ajudar você hoje?'

**Usando com método de template do LangChain**

Para criar prompts dinâmicos e organizados, especialmente em aplicações maiores e reutilizáveis com LangChain, utilizamos `ChatPromptTemplate.from_messages()`, que permite inserir de forma organizada variáveis (como {input}) e separar a lógica do prompt, tornando o código mais limpo e escalável.

Em vez de invocar a LLM diretamente, criamos uma "chain" que combina este template de prompt com o modelo.

> Para contextualizar, o que são **chains**: Chain do LangChain (Corrente, Cadeias ou ainda Sequencias) é uma composição de etapas que processam dados em sequência — aqui, a entrada é formatada pelo prompt e enviada ao modelo. A vantagem é que chains permitem combinar várias ações (como formatar, gerar, filtrar, armazenar) de forma modular e reutilizável, facilitando aplicações mais robustas. Elas funcionam ao encadear componentes, onde a saída de um se torna a entrada do próximo, criando uma sequência lógica de operações.

In [25]:
from langchain_core.prompts import ChatPromptTemplate

prompt = "Olá! quem é você?"  # @param {type:"string"}

template = ChatPromptTemplate.from_messages([
    ("system", "Você é um redator profissional"),
    ("human", "{prompt}")
])

chain = template | llm

res = chain.invoke({"prompt": prompt})
res.content

'Olá! Eu sou um assistente de inteligência artificial treinado para ajudar e fornecer informações sobre uma ampla gama de tópicos. Posso ajudar a responder a perguntas, fornecer definições, explicar conceitos, traduzir textos e muito mais. Estou aqui para ajudar e fornecer assistência de forma clara e objetiva. Como posso ajudar você hoje?'

### Estendendo a chain / Output parser

Para trabalhar com a saída de sequência "crua" da mensagem, Langchain oferece "Output Parsers", como o StrOutputParser, que processa a saída do modelo em um formato mais acessível, convertendo-a em uma string.

Se o modelo (LLM) já produz uma string, o StrOutputParser simplesmente a repassa; se for um ChatModel que produz uma mensagem, ele extrai o conteúdo do atributo `.content`. Embora res.content possa ser usado diretamente no caso de modelos LLM que já retornam string, incluir o StrOutputParser na chain é uma boa prática para obter o valor string diretamente, tornando-se especialmente útil ao integrar ChatModels.



In [26]:
from langchain_core.output_parsers import StrOutputParser

prompt = "Olá! quem é você?"  # @param {type:"string"}

template = ChatPromptTemplate.from_messages([
    ("system", "Você é um redator profissional"),
    ("human", "{prompt}")
])

chain = template | llm | StrOutputParser()

res = chain.invoke({"prompt": prompt})
res

'Olá! Eu sou um assistente digital, um modelo de linguagem treinado por máquina que pode entender e responder a perguntas e solicitações de forma personalizada. Meu objetivo é ajudar e fornecer informações úteis de maneira clara e concisa.\n\nEu não sou um ser humano, mas um programa de computador projetado para simular conversas e fornecer respostas precisas e relevantes. Posso ser usado para uma variedade de tarefas, desde responder a perguntas simples até fornecer ajuda com tarefas mais complexas, como redação de textos, tradução de idiomas e muito mais.\n\nEstou aqui para ajudar e responder às suas perguntas, então sinta-se à vontade para fazer qualquer pergunta ou solicitar ajuda com algo que você precise!'

## Melhorando a exibição do resultado

Note acima que o resultado não ficou tão apresentável no Colab, podemos melhorar a sua visualização usando **Markdown**.
* Markdown é uma linguagem de marcação simples e leve que facilita a formatação de texto usando símbolos como asteriscos e hashtags, sem precisar de HTML. No Google Colab, ele melhora a organização e a legibilidade, permitindo destacar textos em *itálico* , **negrito** e criar títulos com #, ## ou ### para diferentes níveis.

* Caso queira explorar mais, aqui está um guia da sintaxe: https://www.markdownguide.org/basic-syntax/


In [27]:
def show_res(res):
  from IPython.display import Markdown
  display(Markdown(res))

show_res(res)

Olá! Eu sou um assistente digital, um modelo de linguagem treinado por máquina que pode entender e responder a perguntas e solicitações de forma personalizada. Meu objetivo é ajudar e fornecer informações úteis de maneira clara e concisa.

Eu não sou um ser humano, mas um programa de computador projetado para simular conversas e fornecer respostas precisas e relevantes. Posso ser usado para uma variedade de tarefas, desde responder a perguntas simples até fornecer ajuda com tarefas mais complexas, como redação de textos, tradução de idiomas e muito mais.

Estou aqui para ajudar e responder às suas perguntas, então sinta-se à vontade para fazer qualquer pergunta ou solicitar ajuda com algo que você precise!

## Juntando em uma função

Reunir tudo em uma função facilita a reutilização, organização e manutenção do código, evitando repetições durante nossos testes. Vamos chamar essa função no bloco de código seguinte


In [28]:
def llm_generate(llm, prompt):
  template = ChatPromptTemplate.from_messages([
      ("system", "Você é um redator profissional."),
      ("human", "{prompt}"),
  ])

  chain = template | llm | StrOutputParser()

  res = chain.invoke({"prompt": prompt})
  show_res(res)

In [29]:
prompt = "escreva 5 dicas de saúde"  # @param {type:"string"}

llm_generate(llm, prompt)

**Dicas de Saúde: Melhore Sua Qualidade de Vida**

Aqui estão cinco dicas de saúde importantes para ajudá-lo a melhorar sua qualidade de vida e manter seu corpo e mente saudáveis:

1. **Hidrate-se Adequadamente**: Beber água suficiente é essencial para manter o corpo funcionando corretamente. A água ajuda a regular a temperatura corporal, transportar nutrientes e remover toxinas. Tente beber pelo menos 2 litros de água por dia, e aumente a quantidade se você estiver fazendo exercícios ou vivendo em um clima quente.

2. **Alimente-se de Forma Balanceada**: Uma dieta equilibrada é fundamental para manter a saúde. Inclua uma variedade de alimentos em sua dieta, como frutas, legumes, proteínas magras, grãos integrais e laticínios. Evite alimentos processados e ricos em açúcar e gordura, que podem aumentar o risco de doenças crônicas.

3. **Exercite-se Regularmente**: O exercício físico é essencial para manter a saúde cardiovascular, fortalecer os músculos e melhorar a flexibilidade. Tente fazer pelo menos 30 minutos de exercício por dia, como caminhar, correr, nadar ou praticar yoga. Além disso, inclua atividades que você goste, para que o exercício se torne uma parte divertida da sua rotina.

4. **Dê Prioridade ao Sono**: O sono é fundamental para a recuperação do corpo e da mente. Tente dormir entre 7 e 9 horas por noite, e estabeleça uma rotina de sono regular para ajudar a regular o relógio biológico. Evite café e eletrônicos antes de dormir, e crie um ambiente tranquilo e confortável para dormir.

5. **Gerencie o Estresse**: O estresse crônico pode ter um impacto negativo na saúde, aumentando o risco de doenças cardíacas, diabetes e outras condições. Tente encontrar maneiras de gerenciar o estresse, como meditar, praticar yoga, ou fazer atividades que você goste. Além disso, estabeleça limites e priorize as atividades que são importantes para você, para evitar o estresse e a sobrecarga.

Lembre-se de que a saúde é um processo contínuo, e pequenas mudanças podem fazer uma grande diferença ao longo do tempo. Inclua essas dicas em sua rotina e veja como elas podem melhorar sua qualidade de vida!

### Outros Modelos Open Source


 * Modelos disponíveis pelo Groq https://console.groq.com/docs/rate-limits (ver os gratuitos - dentro da aba *free tier*)

 Durante a fase de experimentação é uma boa ideia testar diferentes modelos.

Após validar a nossa solução e fazer os testes iniciais, você pode optar também por modelos pagos e proprietários quando o modelo estiver em produção, já que agora não estará mais desperdiçando alguns centavos de dólar em testes.


### Modelos proprietários (exemplo: ChatGPT da OpenAI)

Recomenda-se iniciar os testes com modelos open source e, após a validação, migrar para soluções pagas, como ChatGPT (OpenAI) ou Claude (Anthropic), para evitar custos desnecessários durante o desenvolvimento.

Soluções pagas oferecem modelos de ponta com alta performance e suporte via API, ideais para robustez e facilidade, mas o custo escala com o uso de tokens (segmentos de texto processados); em contraste, modelos open source, executáveis localmente ou em servidores próprios, proporcionam maior controle, privacidade e custo reduzido para larga escala, exigindo, no entanto, mais conhecimento técnico para configuração e manutenção.

A decisão entre API paga e open source deve considerar o volume de uso esperado e a necessidade de personalização.

Para testar a implementação, faremos um exemplo com o ChatGPT, lembrando que os custos da OpenAI são baseados em tokens (consulte openai.com/api/pricing/).

A grande vantagem de usar LangChain é que toda a sintaxe e lógica de chains criadas são reaproveitáveis, alterando-se apenas a forma como a LLM é carregada, enquanto o restante da aplicação permanece o mesmo.

* Valores: https://openai.com/api/pricing/


> Como gerar uma API key

Para utilizar os modelos da OpenAI, é necessário obter uma chave de API. Siga as etapas abaixo para gerar a sua:

1. Acesse o site da OpenAI e faça login na sua conta.
2. Navegue até a seção de chaves de API e clique em "Criar nova chave secreta" - https://platform.openai.com/api-keys
3. Copie a chave gerada e armazene-a em um local seguro. Importante: nunca compartilhe sua chave

> Conferir o uso https://platform.openai.com/usage

In [30]:
!pip install -q langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.8 MB/s eta 0:00:00


In [31]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [32]:
# https://platform.openai.com/docs/models
from langchain_openai import ChatOpenAI
chatgpt = ChatOpenAI(model = "gpt-4o-mini")

In [33]:
chatgpt = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

In [35]:
chain_chatgpt = template | chatgpt

res = chain_chatgpt.invoke({"prompt": "Gere um texto de 2 parágrafos sobre dicas de saúde"})
show_res(res.content)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

# Construindo o prompt de aplicação

Agora que já interagimos com o modelo de forma básica, vamos começar a explorar a **engenharia de prompt**. Isso significa aprender a formular perguntas ou instruções de forma clara e específica para obter respostas mais precisas e úteis.


### Estrutura de um prompt

Existem várias técnicas de engenharia de prompt, onde muitas delas se baseiam em princípios parecidos. Uma abordagem simples para construir um prompt mais completo é adicionar a ele alguns 'blocos' (componentes), que no caso seriam:
* Papel (Role) - "quem" ele deve interpretar (mais sobre isso abaixo)
* Tarefa (Task) - tarefa que deve realizar
* Entrada (Input) - informação que pode ser usada como contexto para gerar uma resposta (por exemplo o faturamento mensal de uma empresa, ou um dado específico sobre algo ou alguém)
* Saída (Output) - como quer que seja o resultado. Podemos especificar também regras, como como tamanho do resultado (medido em quantidade de palavras ou parágrafos por exemplo)
* Restrições (Constraints) - o que queremos evitar na resposta. Por exemplo: "evite jargões ou linguagem muito técnica". "Não inclua sua análise ou opinião".



### Interpretação de papéis - Role Prompting

A técnica de role prompting consiste em instruir o modelo de IA a assumir um papel específico, como um especialista em determinada área (o modo mais comum, conhecido como "O Especialista", para obter explicações técnicas), uma figura histórica ou um personagem fictício, o que influencia significativamente o estilo e o conteúdo da resposta, mesmo que o restante do prompt seja idêntico.

Utilizar frases como "Você é um redator" ou "Aja como um historiador" no prompt de sistema são exemplos práticos dessa técnica, que molda a persona da LLM para gerar resultados mais alinhados com o contexto desejado.

In [36]:
prompt = "fale sobre chocolate em 1 parágrafo"

template = ChatPromptTemplate.from_messages([
    ("system", "Você é um historiador"),
    ("human", "{prompt}")
])

chain = template | llm | StrOutputParser()

res = chain.invoke({"prompt": prompt})
show_res(res)

O chocolate tem uma rica história que remonta a mais de 3.000 anos atrás, quando os astecas e maias na Mesoamérica consideravam o cacau, a base do chocolate, uma mercadoria valiosa e sagrada. Eles acreditavam que o cacau era um presente dos deuses e o utilizavam em cerimônias religiosas, além de usá-lo como moeda. Com a chegada dos espanhóis, o chocolate foi introduzido na Europa, onde se tornou um luxo acessível apenas à nobreza devido ao seu alto custo. No entanto, com o desenvolvimento de novas técnicas de processamento e a colonização de áreas produtoras de cacau, o chocolate se tornou mais acessível e começou a ser produzido em massa, levando a criação de diversas variedades e tipos de chocolate que conhecemos hoje, desde o amargo e intenso até o doce e cremoso. Hoje em dia, o chocolate é apreciado em todo o mundo, não apenas como um doce delicioso, mas também por seus potenciais benefícios à saúde, como ser rico em antioxidantes e ter propriedades que podem melhorar o humor e a saúde cardiovascular.

Aqui vemos então como o tipo de especialista pode impactar totalmente no resultado. Portanto precisamos pedir algo que seja alinhado ao nosso propósito (copiamos o mesmo código do bloco acima, mudando apenas o prompt do system)

In [37]:
template = ChatPromptTemplate.from_messages([
    ("system", "Você é um especialista em marketing digital."),
    ("human", "{prompt}"),
])

chain = template | llm | StrOutputParser()

res = chain.invoke({"prompt": prompt})
show_res(res)

O chocolate! É um produto que conquistou o coração de muitas pessoas ao redor do mundo. Com sua rica textura e sabor intenso, o chocolate é um alimento que pode ser apreciado de diversas maneiras, seja como um doce, um ingrediente em receitas ou até mesmo como um presente. Além disso, o chocolate também tem benefícios para a saúde, como reduzir o estresse e melhorar o humor, devido à presença de substâncias como a feniletilamina e a teobromina. Com tantas variedades e sabores disponíveis, desde o amargo ao branco, o chocolate é um produto que pode ser personalizado para atender aos mais variados gostos e preferências, tornando-se um dos alimentos mais populares e amados em todo o mundo.

Ser mais específico geralmente é mais indicado. No contexto da nossa aplicação o prompt abaixo funciona melhor porque direciona o modelo não apenas para produzir textos bem escritos, mas com foco estratégico (como conversão, engajamento e SEO, que são essenciais em campanhas de marketing).

In [38]:
template = ChatPromptTemplate.from_messages([
    ("system", "Você é um especialista em marketing digital com foco em SEO e escrita persuasiva."),
    ("human", "{prompt}"),
])

chain = template | llm | StrOutputParser()

res = chain.invoke({"prompt": prompt})
show_res(res)

O chocolate! Um dos prazeres mais universais e deliciosos da vida. Com sua rica textura e sabor intenso, o chocolate é capaz de evocar emoções e sensações únicas em quem o degusta. Desde os finos chocolates artesanais até os doces e cremosos chocolates industriais, existem inúmeras variedades e sabores para explorar. Além disso, o chocolate é também um excelente presente, seja para si mesmo ou para compartilhar com amigos e familiares, e pode ser usado em uma infinidade de receitas, desde bolos e tortas até sobremesas e bebidas. Seja qual for a ocasião, o chocolate é sempre uma escolha saborosa e atraente, capaz de trazer alegria e satisfação a quem o aprecia.

### Usando exemplos - One-Shot e Few-Shot Prompting

 * Zero-Shot – O modelo responde sem exemplos, confiando apenas no treinamento
 * One-Shot – Um exemplo é fornecido para orientar a resposta.
 * Few-Shot – Vários exemplos ajudam o modelo a reconhecer padrões e melhorar a precisão.



In [39]:
assunto = 'chocolate'

one_shot = f"""
Exemplo:
Título: Você sabia que beber mais água pode melhorar sua concentração?
Texto: A desidratação leve já é suficiente para reduzir seu foco e energia no dia a dia. Mantenha uma garrafinha por perto e lembre-se de se hidratar ao longo do dia.
Hashtags: #hidratação #foconasaude

Agora gere um novo texto que fale sobre {assunto}
"""

#print(one_shot)

res = chain.invoke({"prompt": one_shot})
show_res(res)

Título: O Chocolate: Um Doce Amigo para o Seu Bem-Estar!
Texto: Você sabia que o chocolate pode ser benéfico para a saúde? Rico em antioxidantes e flavonoides, o chocolate amargo pode ajudar a melhorar o humor e até mesmo a memória. Então, permita-se um pedaço de chocolate amargo por dia e aproveite os benefícios dessa delícia!
Hashtags: #chocolateamargo #bemestar #saúde

O few-shot prompting, ou prompt com exemplos, demonstra à IA a estrutura, estilo e abordagem desejados para a resposta, como a inclusão de hashtags ou a formulação de títulos como perguntas, tornando o processo de instrução mais intuitivo e eficiente do que apenas fornecer instruções textuais. Embora exemplos possam ser combinados com texto para maior precisão, o few-shot prompting com múltiplos exemplos, como o que veremos a seguir, ajuda a IA a generalizar melhor o padrão esperado.

In [40]:
few_shot = f"""
Exemplo 1:
Título: Você sabia que beber mais água pode melhorar sua concentração?
Texto: A desidratação leve já é suficiente para reduzir seu foco e energia no dia a dia. Mantenha uma garrafinha por perto e lembre-se de se hidratar ao longo do dia.
Hashtags: #hidratação #foconasaude

Exemplo 2:
Título: Comer carboidratos à noite engorda: Mito ou verdade?
Texto: Esse é um mito comum. O que realmente importa é o total calórico do dia e a qualidade dos alimentos. Com orientação certa, dá sim para comer bem à noite sem culpa!
Hashtags: #nutricaosemmitos #equilibrioalimentar

Agora gere um novo texto que fale sobre {assunto}
"""

res = chain.invoke({"prompt": few_shot})
show_res(res)

Título: O Chocolate é o Vilão da Dieta? Descubra a Verdade!
Texto: Muitas pessoas acreditam que o chocolate é um alimento proibido para quem busca uma vida saudável, mas a verdade é que, em moderação, o chocolate pode ter benefícios surpreendentes. O chocolate amargo, por exemplo, é rico em antioxidantes e pode ajudar a melhorar o humor e a saúde cardiovascular. Então, não precisa cortar o chocolate completamente da sua dieta, apenas aproveite com responsabilidade!
Hashtags: #chocolatesaudavel #moderaçãonanutrição

### Guiando o resultado com uma estrutura - Structured Prompting

Para o prompt final de nossa aplicação, usaremos também o conceito de Prompting estruturado (Structured Prompting), cuja premissa envolve a codificação cuidadosa de instruções, exemplos e restrições personalizadas para direcionar propositalmente comportamentos de modelos de linguagem para tarefas de um nicho específicos.



In [41]:
form = create_form()
display(form)

In [42]:
prompt = f"""
Crie um post para {platform.value} com a seguinte estrutura:
1. Comece com uma pergunta provocativa.
2. Apresente um benefício claro relacionado ao tema.
3. Finalize com uma chamada para ação (CTA) encorajando o leitor a buscar mais informações.

Tema: {topic.value}
Público-alvo: {audience.value}
Tom: {tone.value}
"""

print(prompt)


Crie um post para Instagram com a seguinte estrutura:
1. Comece com uma pergunta provocativa.
2. Apresente um benefício claro relacionado ao tema.
3. Finalize com uma chamada para ação (CTA) encorajando o leitor a buscar mais informações.

Tema: 
Público-alvo: Geral
Tom: Normal



In [43]:
res = chain.invoke({"prompt": prompt})
show_res(res)

Você sabia que uma boa estratégia de marketing digital pode aumentar suas vendas em até 500%?

Com uma abordagem certa, você pode alcançar um público mais amplo, aumentar sua visibilidade online e, consequentemente, impulsionar suas vendas. Isso significa mais oportunidades de negócios e um crescimento significativo para sua empresa.

Quer saber como é possível alcançar esses resultados? Clique no link na nossa bio para descobrir mais sobre como uma boa estratégia de marketing digital pode transformar seu negócio! #MarketingDigital #CrescimentoEmpresarial #SucessoOnline

Para dar mais liberdade à IA na escolha da estrutura do texto, especialmente considerando que a aplicação aceitará diversos parâmetros como plataforma e comprimento, optaremos por um prompt final dinâmico em vez de um structured prompting rígido, que seria mais adequado para resultados muito específicos e poderia levar a publicações repetitivas.

### Construindo o prompt final dinamicamente


Este prompt final será construído a partir das variáveis do formulário, organizado em itens legíveis com `-` para fácil modificação e escalabilidade.

Cada linha fornecerá instruções claras (canal, tom, público...), e opções como hashtags ou CTAs serão incluídas condicionalmente usando expressões inline em Python, adaptando o prompt às escolhas do usuário.

Adicionaremos também a instrução para garantir que a saída seja limpa e pronta para uso.



In [44]:
prompt = f"""
Escreva um texto com SEO otimizado sobre o tema '{topic.value}'.
Retorne em sua resposta apenas o texto final.
- Onde será publicado: {platform.value}.
- Tom: {tone.value}.
- Público-alvo: {audience.value}.
- Comprimento: {length.value}.
- {"Inclua uma chamada para ação clara." if cta.value else "Não inclua chamada para ação"}
- {"Retorne ao final do texto hashtags relevantes." if hashtags.value else "Não inclua hashtags."}
{"- Palavras-chave que devem estar presentes nesse texto (para SEO): " + keywords.value if keywords.value else ""}
"""
print(prompt)


Escreva um texto com SEO otimizado sobre o tema ''.
Retorne em sua resposta apenas o texto final.
- Onde será publicado: Instagram.
- Tom: Normal.
- Público-alvo: Geral.
- Comprimento: Curto.
- Não inclua chamada para ação
- Não inclua hashtags.




In [45]:
res = chain.invoke({"prompt": prompt})
show_res(res)

O marketing digital é uma ferramenta essencial para qualquer empresa que deseja se destacar no mercado. Com o uso de técnicas de SEO, é possível aumentar a visibilidade de um site ou página nas buscas online, alcançando assim um público mais amplo. A escrita persuasiva também desempenha um papel fundamental, pois ajuda a engajar e convencer os leitores a adotar uma determinada ação ou adquirir um produto. Juntando essas duas habilidades, é possível criar campanhas de marketing eficazes e alcançar resultados significativos.

### Sobre o prompt e melhorias

Não existe um “melhor prompt” universal — o mais eficaz depende sempre do seu objetivo e do contexto da aplicação. A melhor forma de descobrir o que funciona é testando variações e analisando os resultados.

Para encontrar boas alternativas, você pode:

* Pesquisar por prompt books gratuitos disponíveis na internet

* Usar sites que reúnem templates prontos, como PromptHero ou FlowGPT

* Pedir sugestões diretamente à própria LLM (“Como posso melhorar esse prompt para torná-lo mais persuasivo?”)

* Analisar exemplos de prompts usados em casos reais ou estudos de caso

* Ajustar pequenos trechos do prompt e observar o impacto (tom, foco, estrutura)

* Extra: Combinar técnicas (por exemplo Structured Prompting com few-shot prompting) pode aprimorar ainda mais a qualidade e a relevância dos conteúdos gerados.

Essas estratégias ajudam a refinar continuamente a performance e alinhar melhor o conteúdo gerado aos seus objetivos.

# Concluindo a aplicação final

Agora que concluímos a criação do prompt final de nossa aplicação, podemos partir para a finalização.
Precisamos juntar os formulários ao prompt e à LLM.



In [46]:
def llm_generate(llm, prompt):
  template = ChatPromptTemplate.from_messages([
      ("system", "Você é um especialista em marketing digital com foco em SEO e escrita persuasiva."),
      ("human", "{prompt}"),
  ])

  chain = template | llm | StrOutputParser()

  res = chain.invoke({"prompt": prompt})
  return res

In [47]:
def generate_result(b):
  with output:
    output.clear_output()
    prompt = f"""
    Escreva um texto com SEO otimizado sobre o tema '{topic.value}'.
    Retorne em sua resposta apenas o texto final e não inclua ela dentro de aspas.
    - Onde será publicado: {platform.value}.
    - Tom: {tone.value}.
    - Público-alvo: {audience.value}.
    - Comprimento: {length.value}.
    - {"Inclua uma chamada para ação clara." if cta.value else "Não inclua chamada para ação"}
    - {"Retorne ao final do texto hashtags relevantes." if hashtags.value else "Não inclua hashtags."}
    {"- Palavras-chave que devem estar presentes nesse texto (para SEO): " + keywords.value if keywords.value else ""}
    """
    try:
      res = llm_generate(llm, prompt)
      show_res(res)
    except Exception as e:
      print(f"Erro: {e}")

Para executar a função de geração de conteúdo ao clicar no botão, precisamos primeiro desvincular qualquer callback anterior para evitar execuções duplicadas, especialmente em ambientes como o Colab onde o parâmetro remove=True pode apresentar instabilidades. A solução mais simples e robusta é redeclarar o output, o generate_button (associando o on_click à nova função) e a variável form chamando create_form(), garantindo uma configuração limpa a cada execução da célula.

In [ ]:
output = widgets.Output()
generate_button = widgets.Button(description = "Gerar conteúdo")
generate_button.on_click(generate_result)
form = create_form()

In [ ]:
display(form)

**Pronto!** Finalizamos nossa aplicação.

Aqui você pode reunir todo o código desenvolvido em um único bloco, já pronto para ser executado e utilizado por quem for operar o sistema.

Para deixar o código recolhido por padrão, utilize o comando `#@title` no início do bloco — por exemplo: `#@title Rodar Aplicação`
Isso além de criar uma seção com título e facilitar a organização vai permitir que o código fique escondido. Para exibir ou ocultar o conteúdo, basta dar dois cliques sobre o título ("Rodar Aplicação").

## Escalando para outras áreas e adicionando mais campos

Para aumentar a flexibilidade na definição das opções dos campos Dropdown, em vez de fixá-las no código, utilizaremos os formulários do Colab com a anotação `@param {type:"string"}`. Isso permite que o usuário insira uma lista de valores separados por vírgula diretamente em um campo ao lado da célula de código, que é então convertida em uma lista Python e usada dinamicamente no parâmetro options do widget.

Dessa forma, o formulário se torna totalmente configurável, permitindo fácil adição ou modificação das opções dos dropdowns, como as do campo "comprimento", sem alterar o código principal.


In [ ]:
opt_length = "Curto, Médio, Longo, 1 parágrafo, 1 página" # @param {type:"string"}
print(opt_length)

Curto, Médio, Longo, 1 parágrafo, 1 página


In [ ]:
options_length = [x.strip() for x in opt_length.split(",")]

In [ ]:
options_length

['Curto', 'Médio', 'Longo', '1 parágrafo', '1 página']

In [ ]:
length = widgets.Dropdown(
    options = option_length,
    description="Tamanho",
    layout=widgets.Layout(width=w_dropdown)
)

In [ ]:
form = create_form()
output.clear_output()
display(form)


---

## Construção de interface com Streamlit

Após validar que nossa aplicação está funcionando corretamente, podemos aprimorar ainda mais a interface.

Embora o uso de ipywidgets pode ser funcional, conseguimos criar uma experiência mais amigável e visual com o **Streamlit** — uma ferramenta focada em interfaces interativas para aplicações em Python. Além disso, o Streamlit facilita o deploy da aplicação, tornando-a mais acessível para equipes de atendimento ou até mesmo clientes finais.





### 1. Instalação do Streamlit

Para começarmos, precisamos instalar o **Streamlit**

Por estarmos rodando no Colab, precisa também instalar o **Localtunnel** para conseguirmos nos conectar à aplicação gerada com o streamlit. Ao executar em seu próprio computador ela não é necessária, pois após rodar o comando de launch do streamlit ("streamlit run ...") será aberto automaticamente uma aba em seu navegador com a aplicação.

Além disso, vamos instalar a biblioteca **dotenv**, usada para simplificar a gestão de variáveis de ambiente ao armazená-las em um arquivo .env.





In [ ]:
!pip install -q streamlit
!npm install -q localtunnel
!pip install -q python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.8 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
added 22 packages in 5s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸

### 2. Criação do arquivo da aplicação

Crie um arquivo chamado `app.py` (ou outro nome que preferir) com o conteúdo do seu código adaptado para Streamlit.

Antes de colocarmos o código nesse arquivo, vamos criar o arquivo .env, para carregar as variáveis de ambiente. Aqui basta colocarmos a key do Groq, a mesma que usamos anteriormente. Deixe nesse formato: `GROQ_API_KEY=CHAVE_AQUI`

* Obs: o comando `%%writefile` no início desse bloco de código permite que a célula do notebook seja salva como um arquivo externo, com o nome especificado. Ou seja, estamos criando um arquivo com esse nome e o conteúdo será tudo a partir da segunda linha do bloco abaixo


In [ ]:
%%writefile .env
GROQ_API_KEY=#######

Writing .env


Foi necessário fazer algumas adaptações ao código, pois até então usamos ipywidgets mas agora no Streamlit usaremos funções da própria biblioteca para criar os campos.

In [ ]:
%%writefile app.py
import streamlit as st
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
load_dotenv()

## conexão com a LLM
id_model = "llama3-70b-8192"
llm = ChatGroq(
    model=id_model,
    temperature=0.7,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

## função de geração
def llm_generate(llm, prompt):
  template = ChatPromptTemplate.from_messages([
      ("system", "Você é um especialista em marketing digital com foco em SEO e escrita persuasiva."),
      ("human", "{prompt}"),
  ])

  chain = template | llm | StrOutputParser()

  res = chain.invoke({"prompt": prompt})
  return res

st.set_page_config(page_title = "Gerador de conteúdo 🤖", page_icon="🤖")
st.title("Gerador de conteúdo")

# Campos do formulário
topic = st.text_input("Tema:", placeholder="Ex: saúde mental, alimentação saudável, prevenção, etc.")
platform = st.selectbox("Plataforma:", ['Instagram', 'Facebook', 'LinkedIn', 'Blog', 'E-mail'])
tone = st.selectbox("Tom:", ['Normal', 'Informativo', 'Inspirador', 'Urgente', 'Informal'])
length = st.selectbox("Tamanho:", ['Curto', 'Médio', 'Longo'])
audience = st.selectbox("Público-alvo:", ['Geral', 'Jovens adultos', 'Famílias', 'Idosos', 'Adolescentes'])
cta = st.checkbox("Incluir CTA")
hashtags = st.checkbox("Retornar Hashtags")
keywords = st.text_area("Palavras-chave (SEO):", placeholder="Ex: bem-estar, medicina preventiva...")

if st.button("Gerar conteúdo"):
  prompt = f"""
  Escreva um texto com SEO otimizado sobre o tema '{topic}'.
  Retorne em sua resposta apenas o texto final e não inclua ela dentro de aspas.
  - Onde será publicado: {platform}.
  - Tom: {tone}.
  - Público-alvo: {audience}.
  - Comprimento: {length}.
  - {"Inclua uma chamada para ação clara." if cta else "Não inclua chamada para ação"}
  - {"Retorne ao final do texto hashtags relevantes." if hashtags else "Não inclua hashtags."}
  {"- Palavras-chave que devem estar presentes nesse texto (para SEO): " + keywords if keywords else ""}
  """
  try:
      res = llm_generate(llm, prompt)
      st.markdown(res)
  except Exception as e:
      st.error(f"Erro: {e}")

Overwriting app.py


### 3. Execução do Streamlit

Tendo nosso script pronto, basta executar o comando abaixo para rodar a nossa aplicação pelo streamlit.
Isso fará com que a aplicação do Streamlit seja executada em segundo plano.

In [ ]:
!streamlit run app.py &>/content/logs.txt &

> **Como abrir a interface**

> Importante: caso esse código não funcione corretamente use o ngrok, cujo código você encontra mais abaixo (para mais detalhes, veja a aula 'Aviso sobre uso no Colab')

* Antes de conectar com o localtunnel, você precisa obter o IP externo (usando esse comando `!wget -q -O - ipv4.icanhazip.com`). Copie esse número, que vai aparecer na saída do bloco abaixo (após rodar)
* Então, entre no link que aparece na saída do bloco abaixo e informe esse IP no campo Tunnel Password. Logo em seguida, clique no botão e aguarde a interface ser inicializada


Esse comando usa npx localtunnel para "expor" o aplicativo Streamlit em execução local para a internet. O aplicativo é hospedado na porta 8501, e o localtunnel fornece uma URL pública por meio da qual o aplicativo pode ser acessado.

**Caso não abra, reinicie a sessão e espere alguns segundos antes de clicar no link. Ou, reinicie o ambiente de execução e rode os comandos novamente.**

In [ ]:
!wget -q -O - ipv4.icanhazip.com
!npx localtunnel --port 8501

34.143.132.173
⠙your url is: https://witty-planets-laugh.loca.lt


> **Importante:** Caso o comando acima com localtunnel não funcione, use o código abaixo (Para mais detalhes, consulte a aula "Aviso sobre uso no Colab" da seção 2)

### Alternativa com ngrok

In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok

!ngrok config add-authtoken SEU_TOKEN_AQUI
!streamlit run app.py --server.port 8501 &>/content/logs.txt &

public_url = ngrok.connect(8501)
public_url

---

## Rodando a LLM localmente

Se for um modelo open source nós podemos fazer o download e rodar localmente em um provedor cloud (como nesse caso o colab) ou em nosso próprio computador.






### -> Para executar no Colab

**Importante:** Antes de realizar os próximos passos, mude o ambiente de execução no Colab para usar GPU, que será necessário já que todo o processamento será feito direto localmente no ambiente de execução do Colab. Para isso, selecione 'Ambiente de execução > Alterar o tipo de ambiente de execução' e na opção 'Acelerador de hardware' selecione 'GPU'.

Além das bibliotecas do langchain que instalamos, vamos precisar também da biblioteca `langchain-huggingface`, `transformers` e `bitsandbytes`

In [ ]:
!pip install -q langchain langchain-community langchain-huggingface transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.2 MB/s eta 0:00:00


In [ ]:
!pip install bitsandbytes-cuda110 bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.6 MB/s eta 0:00:00


In [ ]:
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
import torch

**Quantização**

A execução de LLMs pode ser desafiadora devido aos recursos limitados, especialmente na versão gratuita do Google Colab. Para contornar essa limitação, além de escolher modelos com menos parâmetros podemos usar técnicas de quantização, como o `BitsAndBytesConfig` da biblioteca `transformers`, que permitem carregar e executar modelos massivos de forma eficiente sem comprometer significativamente o desempenho.
* Essas técnicas reduzem os custos de memória e computação ao representar pesos e ativações com tipos de dados de menor precisão, como inteiros de 8 bits (int8) ou até 4 bits, tornando viável o uso de modelos grandes mesmo em hardware limitado.
* Alternativas ao BitsAndBytesConfig: AutoGPTQ, AutoAWQ, etc.
* Para quem prefere evitar configurações complexas de otimização e manter a máxima qualidade, considere o uso via API.
* Mais detalhes sobre quantização: https://huggingface.co/blog/4bit-transformers-bitsandbytes

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

**Download do modelo**

Agora faremos o download e a configuração de um modelo do HuggingFace usando o método `AutoModelForCausalLM.from_pretrained`. Este processo pode levar alguns minutos, pois o modelo tem alguns GB - mas no geral o download no Colab deve ser relativamente rápido.

> Para ver todos os modelos disponíveis no Hugging Face, acesse: https://huggingface.co/models?pipeline_tag=text-generation

Escolhemos o Phi 3 (microsoft/Phi-3-mini-4k-instruct), um modelo menor mas que demonstrou ser muito interessante e com ótimo custo benefício
 - https://huggingface.co/microsoft/Phi-3-mini-4k-instruct



In [ ]:
model_id = "microsoft/Phi-3-mini-4k-instruct"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

**Criação do Pipeline**

Agora criaremos um pipeline para geração de texto usando nosso modelo e tokenizer carregados anteriormente. A função de pipeline HuggingFace simplifica o processo de execução de várias tarefas de processamento de linguagem natural ao fornecer uma interface de alto nível.

Parâmetros:
* `model`: Modelo de linguagem a ser usado (definido por model_id).

* `tokenizer`: Tokenizador correspondente ao modelo para processar o texto.

* `task`: Tipo de tarefa (ex.: "text-generation" para geração de texto).

* `temperature`: Controla a aleatoriedade (lembre-se de variar o valor, conforme as dicas que passamos).

* `max_new_tokens`: Número máximo de tokens gerados na saída.

* `do_sample`: Habilita/desabilita amostragem estocástica (geração não determinística).

* `repetition_penalty`: Penaliza repetições (valores >1 reduzem repetições).

* `return_full_text`: Se False, retorna apenas o texto gerado (ignorando o prompt).

In [ ]:
pipe = pipeline(
    model = model,
    tokenizer = tokenizer,
    task = "text-generation",
    temperature = 0.1,
    max_new_tokens = 500,
    do_sample = True,
    repetition_penalty = 1.1,
    return_full_text = False
)

Device set to use cuda:0


Para carregar a LLM

In [ ]:
llm = HuggingFacePipeline(pipeline = pipe)

In [ ]:
input = "Gere um texto sobre alimentação saudável, em 1 parágrafo"

**Geração do resultado**

In [ ]:
output = llm.invoke(input)
print(output)

.

Texto: A alimentação é fundamental para manter o corpo e a mente funcionando de forma ideal. Uma dieta equilibrada deve incluir uma variedade de nutrientes essenciais que nos permitem crescer, se recuperar após atividades físicas e desempenhar nossas funções cognitivas com maior eficiência. Comidas ricas em fibras como frutas, legumes e grãos integrais ajudam na digestão e no controle do peso. Proteínas encontradas em carnes magras, laticínios ou alternativas vegetais são cruciais para reparar tecidos danificados e construir novos. Vitaminas e mineraños obtidos por meio da ingestão adequada de frutas frescas, verduras folhosas e peixes podem fortalecer o sistema imunológico e prevenir deficiências. Além disso, hidratação constante é vital; água serve não apenas para transportar os nutrientes ao longo dos sistemas corporais, mas também regula a temperatura interna e elimina resíduos toxicos. Em suma, escolher opções alimentares equilibradas apoia tanto a longevidade quanto a qualidad

**Adequando o prompt com Templates (quando necessário)**

Talvez o resultado acima ficou um pouco estranho, ou ele inventou algum texto antes de fornecer o resultado.
Para evitar alucinações ou geração infinita de texto, use o template oficial do modelo Phi 3, que inclui tokens especiais como:

* <|system|>, <|user|>, <|assistant|>: definem os papéis da mensagem.

* <|end|>: marca o fim do texto (equivalente ao token EOS).

Na dúvida, acesse a página do modelo no Hugging Face, se houver um template recomendável para o modelo ele estará na descrição.

Para outras implementações pode não ser necessário fornecer o prompt, como por exemplo a implementação via API que usamos anteriormente.



In [ ]:
prompt = """
<|system|>
Você é um especialista em marketing digital com foco em SEO e escrita persuasiva.<|end|>
<|user|>
"{}"<|end|>
<|assistant|>
""".format(input)

In [ ]:
prompt

'\n<|system|>\nVocê é um especialista em marketing digital com foco em SEO e escrita persuasiva.<|end|>\n<|user|>\n"Gere um texto sobre alimentação saudável, em 1 parágrafo"<|end|>\n<|assistant|>\n'

In [ ]:
output = llm.invoke(prompt)
output

' A alimentação saudável não se trata apenas de escolher opções nutritivas; ela representa uma abordagem holística para a vida que beneficia tanto o corpo quanto a mente. Com cada bocado de fruta fresca ou legume crucifero, estamos nos aproximando da essência do bem-estar geral. Esses alimentos ricos em vitaminas, minerais e fibras são os pilares fundamentais para manter nossa energia constante ao longo do dia e fortalecer nossos sistemas imunológicos contra as adversidades invisíveis. Além disso, praticar hábitos alimentares conscientes pode ser visto como uma forma de autoajuda, onde cada decisão alimentar reflete no nosso estado mental e físico. Portanto, priorizarmos a qualidade dos alimentos consumidos é reconhecer sua importância vital na construção de uma vida plena e equilibrada.'

* Considerações finais: A vantagem de usarmos o LangChain é que toda a sintaxe e lógca que criamos para esse projeto (por exemplo chains) é reaproveitada, o que muda é a parte de carregar a llm, o resto pode permanecer igual. Então, bastaria substituir o método de carregamento da LLM do LangChain (por exemplo, ao invés de ChatGroq usar o HuggingFacePipeline ou o ChatHuggingFace) e com isso você teria a aplicação funcionando o mesmo modo, porém rodando tudo localmente (seja cloud ou no computador local)

### -> Para rodar em seu computador

Para usar a LLM localmente via API: use o mesmo código desse Colab, fazendo a instalação das bibliotecas instaladas (no comando de instalação, ao início desse Colab).

Para usar a LLM baixando o modelo localmente:
Para maior compatibilidade de execução de LLMs em máquina local nós sugerimos a biblioteca [Ollama](https://ollama.com), que possui integração direta com o LangChain.

* Rode o arquivo llm_local.py e instale todas as bibliotecas necessárias conforme consta nos comentários ao início do .py

Recomendamos usar pelo Colab pelo menos no início e para não atrapalhar o fluxo de aprendizado deste curso. Ao executar localmente podem ocorrer outros problemas de instalação ou incompatibilidade, e de início pode perder tempo desnecessário. O método que mostraremos tenta evitar esses tipos de erro mas ainda assim é impossível garantir 100%, portanto sugerimos primeiro testar pelo Colab e depois (se quiser) executar em sua máquina local.

